In [1]:
from src.pispec import PiSpec
from src.trace_utils import TraceParams
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
import time
from itertools import product
import re
import datetime


# initialize the interface0 v
pispec = PiSpec()
print(pispec.tracecontroller.device)
# TODO: this should be in config files


# alternate the datapoints between two detectors? 
# Thats a tracecontroller things

# the tracecontroller is broken

# pispec.tracecontroller._debug()



waiting for serial... 0.0005012297630310058
/dev/ttyACM1


In [2]:
# pispec.actinic_test(int_values=[10, 50, 100, 200])

In [3]:
# pispec.meas_led_test()

In [4]:
# for i in range(0, 100):
#     pispec.run_experiment(exp_name="node", wavelengths=['830'], act_phase_vals=[0, 0, 0], btwn_trace_delay=1)
#     print(i)

## Experiment Test

In [5]:
# exp_name = "leaftest2"
# wavelengths = ["830"]
# act_phase_vals = [0, 200, 0]

# values = ["zero_val", "val", "raw_diff", "V", "d_abs"]

# results = pispec.run_experiment(exp_name=exp_name, wavelengths=wavelengths, act_phase_vals=act_phase_vals)
# print(results)



# df = pispec.get_df()
# print(df.head)

# dest = pispec.save_df(df)
# print(dest)

## artifact experiment

Plant/ClearDetectorPath/ML/SP
1. 1111
2. 1101
3. 1001

In [6]:
pispec = PiSpec()
reps = 10
# pispec = PiSpec()

exp_params = {"1001": {"exp_name": "1001",
                    "wavelengths":["none"],
                    "act_phase_vals": [0, 200, 0]},
              "1111": {"exp_name": "1111",
                        "wavelengths":["830"],
                        "act_phase_vals": [0, 200, 0]},
              "1101": {"exp_name": "1101",
                    "wavelengths":["none"],
                    "act_phase_vals": [0, 200, 0]},

                }

# 1111
for design, rep in product( [key for key in exp_params.keys()], [i for i in range(reps)]):
    print(rep, design)
    exp_name = f'{exp_params[design]["exp_name"]}-{rep}'
    wavelengths = exp_params[design]["wavelengths"]
    act_phase_vals = exp_params[design]["act_phase_vals"]

    print(exp_name, wavelengths, act_phase_vals)

    results = pispec.run_experiment(
        exp_name=exp_name, 
        wavelengths=wavelengths, 
        act_phase_vals=act_phase_vals)

    df = pispec.get_df()
    dest = pispec.save_df(df)
    time.sleep(20)
    if rep == reps - 1 and design == "1001":
        blocked = input('Please unblock the detector. \n')

    



waiting for serial... 0.0005009212493896485
0 1001
1001-0 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/221216_1001-0, True
1 1001
1001-1 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/221216_1001-1, True
2 1001
1001-2 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/221216_1001-2, True
3 1001
1001-3 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/221216_1001-3, True
4 1001
1001-4 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/221216_1001-4, True
5 1001
1001-5 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/221216_1001-5, True
6 1001
1001-6 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/221216_1001-6, True
7 1001
1001-7 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/221216_1001-7, True
8 1001
1001-8 ['none'] [0, 200, 0]
os.path.exists: /home/pi/projects/pi-spec-cli/export/2212

## Plot the data

In [ ]:

pts = df.filter(regex = r'p_|z_|time_us', axis=1)[0:-1]
pts.head()
pts.tail()


In [ ]:
subdf = df.loc[df["nm"] == nm]
subdf.plot(
    x="time_ms",
    y=,
    kind="scatter",
    c="cornflowerblue",
    title=f"{nm}nm {col} vs time_ms",
    suptitle=f"{title}",
    ylabel=f"{col}",
    xlabel="time (ms)",
    s=10,
)


from 100-190, relative signal
before saturation pulse
delat transmission, change relative to the pre sat pulse transmission

deltaT / T

T = pre saturation pulse

THEN 

dA = (- deltaT/T)/2.3


In [ ]:
# import re

# resp = re.findall(pattern="v[0-9]", string=df.param_string[0])
# print(resp)

# divide 

In [ ]:
# # row means of all the data points
# df["val"] = df[['aq_0', 'aq_1', 'aq_2', 'aq_3', 'aq_4']].mean(numeric_only = True, axis=1)

# df["V"] = df["val"] / 6535


In [ ]:
# plt.plot(df['time_us'], df['V'])
# plt.ylim(0, 10)
# plt.show()

In [ ]:
# (x='time_us', y='val')
# plt.show()

# plt.plot(df['time_us'], df['aq_0'])
# plt.show()

In [ ]:

# pispec.tracecontroller.modify_actinic(intensity=500)
# time.sleep(5)
